# 0.0 Imports

In [1]:
import pandas as pd

# 0.1 Functions

# 1 Data Description

## 1.1 Data Extraction and copy

In [8]:
# Data extraction
df = pd.read_csv('amz_br_total_products_data_processed.csv')

# Copy of dataset in order to preserve raw data
df1 = df.copy()




## 1.2 Cols Names Transformation

In [ ]:
# Cols names transformation (if necessary) - No need to transform columns names as they are already in camel case


## 1.3 Dataset Dimensions

In [9]:
# Data dimensions (shape of df)
print(f'The number of columns in the dataset is:', df1.shape[0])
print(f'The number of columns in the dataset is:', df1.shape[1])


The number of columns in the dataset is: 1344513
The number of columns in the dataset is: 11


## 1.4 - Data Types Observation

In [10]:
df1.dtypes

asin                  object
title                 object
imgUrl                object
productURL            object
stars                float64
reviews                int64
price                float64
listPrice            float64
categoryName          object
isBestSeller            bool
boughtInLastMonth      int64
dtype: object

## 1.5 Check NA

In [12]:
df1.isna().sum()

asin                 0
title                1
imgUrl               0
productURL           0
stars                0
reviews              0
price                0
listPrice            0
categoryName         0
isBestSeller         0
boughtInLastMonth    0
dtype: int64

## 1.6 Fill NA

In [15]:
# No need to filL NA's as the dataset do not have any.


## 1.7 Data Types Transformation

In [16]:
# No need to change data types. The dataset already has tem set correctly.



## 1.8 Descriptive Stats

### 1.8.1 Numerical Variables

In [34]:
# Surpassing exponantial notation into 2 decimals format
pd.options.display.float_format = '{:.2f}'.format

num_var = df1.select_dtypes(include=['float64', 'int64'])
num_var.describe()

,stars,reviews,price,listPrice,boughtInLastMonth
count,1344513.00,1344513.00,1344513.00,1344513.00,1344513.00
mean,1.47,212.23,217.86,65.30,4.35
std,2.11,4581.19,856.21,2874.17,94.36
min,0.00,0.00,0.00,0.00,0.00
25%,0.00,0.00,25.00,0.00,0.00
50%,0.00,0.00,69.00,0.00,0.00
75%,4.20,3.00,153.99,0.00,0.00
max,5.00,1878552.00,100000.00,1999000.00,40000.00


### 1.8.1 Categorical Variables

In [35]:
cat_var = df1.select_dtypes(exclude=['float64', 'int64'])

# Finding unique categories for each categorical variable
cat_var.apply(lambda x: x.unique().shape[0])

asin            1344513
title           1308169
imgUrl          1168557
productURL      1344513
categoryName        206
isBestSeller          2
dtype: int64

# 2 Hypothesis Map 

In [41]:
# Client view
    # Categories with higher stars sell more
    # Categories with more reviews sell more
    # Categories best seller sell more
    # Categories bouth in last month sell more

# Price view
    # How the price of products is divided by category?
    # How the price of products is divided by range of price?
    # Products with listprice sell more

# 3 EDA

In [42]:
# Copy of dataset 
df2 = df1.copy()
df2.head()

,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,categoryName,isBestSeller,boughtInLastMonth
0,B0C154VR2L,"SAMSUNG Smart TV Crystal 50"" 4K UHD CU7700 - A...",https://m.media-amazon.com/images/I/71vk2qFDSP...,https://www.amazon.com.br/dp/B0C154VR2L,4.60,269,2279.05,0.00,"TV, Áudio e Cinema em Casa",False,0
1,B0C85J5HYD,Mini Projetor Portatil 5G Wifi 6 Bluetooth 5.0...,https://m.media-amazon.com/images/I/5133wKaOqS...,https://www.amazon.com.br/dp/B0C85J5HYD,4.50,816,409.99,0.00,"TV, Áudio e Cinema em Casa",True,5000
2,B0C1538ZJ4,"Samsung Smart TV Crystal 43"" 4K UHD CU7700 - A...",https://m.media-amazon.com/images/I/71vk2qFDSP...,https://www.amazon.com.br/dp/B0C1538ZJ4,4.70,638,1969.00,2499.00,"TV, Áudio e Cinema em Casa",False,0
3,B0B4V379PC,"Samsung 60BU8000 - Smart TV LED 60' 4K UHD, Wi...",https://m.media-amazon.com/images/I/8110fzpU-P...,https://www.amazon.com.br/dp/B0B4V379PC,4.60,675,3099.00,0.00,"TV, Áudio e Cinema em Casa",False,0
4,B0899BRB4B,Smart TV LED 32'' HD Samsung LH32BETBLGGXZD,https://m.media-amazon.com/images/I/61M025WzQd...,https://www.amazon.com.br/dp/B0899BRB4B,4.70,3945,1079.90,1369.00,"TV, Áudio e Cinema em Casa",True,0


In [ ]:
# Multivariada Analysis
# Bivariada Analysis
# Hiphotesys Validation
